# Recursive implementation of FSAs

This expansion unit doesn't teach you anything new, but it gives you another opportunity to see recursive functions in action.
The usual disclaimers for recursive functions apply:

1. Python has a very limited implementation of recursive functions, so don't use them if you can avoid it.
1. Recursive functions can be very inefficient if you don't use memoization to avoid recomputing results.
1. Despite all that, recursive functions are an essential programming tool, and in some languages they fully replace `for`-loops and `while`-loops.
   So try to master them even if they don't pay off for Python.
   
Alright, with that out of the way, let's look at a recursive version of our `accepts` function for FSAs.
As in the first notebook of this unit, we do not use classes.
But we will assume that an FSA is a dictionary and that the FSA transitions are represented as nested dictionaries.

In [ ]:
def rec_accept(sentence, fsa, cs):
    """A recursive implementation of accept."""
    # base case: sentence is empty
    if not sentence:
        return cs in fsa["F"]
    # recursion step
    else:
        word = sentence[0]
        ns = fsa["T"].get(cs, {}).get(word)
        return rec_accept(sentence[1:], fsa, ns) if ns else False
    
    
def accepts(sentence, fsa):
    """Shorthand for calling rec_accept with inital state."""
    return rec_accept(sentence, fsa, fsa["I"])

First let's make sure that this function does what we expect for our toy FSA from the first notebook.

In [ ]:
fsa = {"I": 1,
       "F": {4},
       "T": {1: {"John": 2},
             2: {"likes": 3},
             3: {"Bill": 4}
            }
      }

In [ ]:
sentence = ["John", "likes", "Bill"]
accepts(sentence, fsa)

In [ ]:
sentence = ["John", "likes", "Sue"]
accepts(sentence, fsa)

Alright, time to take a gander at how the recursive function gets these results.
Are you ready?
This will take a while, so take a deep breath.

We start with `sentence = ["John", "likes", "Bill"]`:

1.  **First instance**
    1. When we call `accepts(sentence, fsa)`, this immediately calls `rec_accept(sentence, fsa, fsa["I"])` instead.
       In other words, we start the recursive function with the initial state as our current state.
    1. Since `sentence` is not empty, `rec_accept` looks at the first item of `sentence`, which is `"John"`.
       It then consults the transition dictionary to see what state is reached by following `"John"` from the current state `1`.
       The answer is `2`.
    1. Since the next state is defined, `rec_accept` calls itself, but with changed arguments:
        - Instead of `sentence`, we pass `sentence[1:]`.
          So `"John"` has been clipped off, leaving only `["likes", "Bill"]`.
        - The variable `fsa` does not change at all.
        - The current state is set to `2`.
1.  **Second instance**  
    This starts the second instance of `rec_accept`.
    Let's call it `rec_accept<2>` here to distinguish it from the previous instance, `rec_accept<1>`.
    1. Again `sentence` is not empty, so `rec_accept<2>` looks at its first item, i.e. `"likes".
       According to `fsa`, the next state is now `3` since that's where `"likes"` takes us from `2`.
    1. Since the next state is defined, we call the third instance of `rec_accept`, which we will call `rec_accept<3>`.
       We again modify the arguments:
        - Instead of `sentence`, we pass `sentence[1:]`.
          But keep in mind that this is already the truncated version of `sentence`.
          So `"likes"` has been clipped off from `["likes", "Bill"]`, leaving only `["Bill"]`.
        - The variable `fsa` does not change at all.
        - The current state is set to `3`.
1.  **Third instance**
    1. Once again `sentence` is not empty, so `rec_accept<3>` looks at its first item, which is `"Bill"`.
       From state `3`, this will take us to `4`.
    1. As before, the next state is defined so we call `rec_accept` one more time with modified arguments:
        - Instead of `sentence`, we pass `sentence[1:]`.
          But keep in mind that this is already the truncated version of `sentence`.
          So `"Bill"` has been clipped off from `["Bill"]`, leaving only `[]`.
        - The variable `fsa` does not change at all.
        - The current state is set to `4`.
1.  **Fourth instance**
    1. Since `sentence` is empty, we check if `4` is a final state.
       This is indeed the case, so `rec_accept<4>` returns `True`.
    1. Since `rec_accept<3>` returns whatever `rec_accept<4>` returns, it now returns `True`.
    1. Since `rec_accept<2>` returns whatever `rec_accept<3>` returns, it now returns `True`.
    1. Since `rec_accept` returns whatever `rec_accept<2>` returns, it now returns `True`.
    1. Since `accept` returns whatever `rec_accept` returns, it now returns `True`.

Got that?
Easy, right?
Okay, time to look at the illicit counterpart `["John", "likes", "Sue"]`.

1.  **First instance**  
    same as before
1.  **Second instance**  
    same as before
1.  **Third instance**
    1. Once again `sentence` is not empty, so `rec_accept<3>` looks at its first item, which is `"Sue"`.
       Since there is no such transition from state `3`, the next state is `None`.
    1. As a result, `rec_accept<3>` returns `False`.
    1. Since `rec_accept<2>` returns whatever `rec_accept<3>` returns, it now returns `False`.
    1. Since `rec_accept` returns whatever `rec_accept<2>` returns, it now returns `False`.
    1. Since `accept` returns whatever `rec_accept` returns, it now returns `False`.

So there you have it.
Intuitively, the recursive function worms its way through `sentence`, word by word, until it either reaches the end or there are no longer any possible transitions.
In the latter case, it always returns `False`.
In the former, it depends on whether the last state the function computed is a final state of the FSA.

## Bullet point summary

- At the risk of constant repetition, recursive functions should be avoided in Python.
- Equally at the risk of constant repetition, a good programmer must be able to write recursive functions.
  They are an essential programming tool, and there are languages like *Haskell* or *OCaml* that rely almost exclusively on recursive functions for iteration.